In [1]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
# load the DICOM files
dcm_path = './dicom-files/arq3/pt2'

files = []

for f in os.listdir(dcm_path):
    filename = os.path.join(dcm_path, f)
    if filename[-3:] == 'dcm':
        files.append(pydicom.dcmread(filename))

In [3]:
# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)


In [4]:
# Creating 3D representation of scan with the slice set
bit_deph = slices[0].BitsStored
print(bit_deph)
rep3d_shape = [*slices[0].pixel_array.shape, len(slices)]
rep3d = np.zeros(rep3d_shape)
for idx, slice in enumerate(slices):
    img = slice.pixel_array.copy()
    img *= (1 << 4)
    rep3d[:, :, idx] = img

resize_factor = 512/rep3d_shape[-1]

12


/home/rafael/Workspace/PDI-Medicas/Rafael/artigo/venv/lib/python3.11/site-packages/pydicom/pixel_data_handlers/pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (11-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


In [5]:
idx = 150
pv0 = np.array(rep3d[:,:,int(idx/resize_factor)], dtype=np.uint16)
pv1 = np.array(rep3d[:,idx,:], dtype=np.uint16)
pv2 = np.array(rep3d[idx,:,:], dtype=np.uint16)
pv1 = cv2.resize(pv1, (512, 512), interpolation=cv2.INTER_AREA)
pv2 = cv2.resize(pv2, (512, 512), interpolation=cv2.INTER_AREA)

In [6]:
pv0.shape, pv1.shape, pv2.shape

((512, 512), (512, 512), (512, 512), (512, 512))

In [19]:
mask = cv2.inRange(pv0, 20480, 1 << 16 -1)
cv2.imshow('window', cv2.bitwise_and(pv0, pv0, mask = mask))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# for ini in range(0, 1 << 16 - 1, 2048):
#     print(ini)
#     mask = cv2.inRange(pv2, ini, 1 << 16 - 1)
#     cv2.imshow('window', mask)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [ ]:

# cv2.imshow('window', pv1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.imshow('window', pv2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()